# Skript für den Jetbot der Roadfollowing und Collision Avoidance kombiniert

In [1]:
import math
import os
import time
from uuid import uuid1

import PIL.Image
import ipywidgets as widgets
import torch
import torch.nn.functional as functional
import torchvision.transforms as transforms
import traitlets
from IPython.display import display
from torch2trt import TRTModule

from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

from elements.yolooo import OBJ_DETECTION

In [2]:
Object_classes = ['sign_forbidden', 'sign_limit', 'sign_nolimit', 'sign_stop', 'kreuzung']
Object_detector = OBJ_DETECTION('weights/best_n.pt', Object_classes)


device = torch.device('cuda')

model_road = TRTModule()
model_road.load_state_dict(torch.load('weights/best_steering_model_xy_17_04_a_trtv2.pth'))

model_collision = TRTModule()
model_collision.load_state_dict(torch.load('weights/collision_model_resnet_19_04_a_trt.pth'))

Model summary: 157 layers, 1765930 parameters, 0 gradients


<All keys matched successfully>

Preprocess Image

In [3]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

Create Camera instance

In [4]:
camera = Camera.instance(width=224, height=224, fps=10)

Widget for information and camera feed display

In [5]:
image_widget = widgets.Image()

speed_output = widgets.Text(
    value='',
    placeholder='',
    description='(l,r):',
    disabled=True
)

model_output = widgets.Textarea(
    value='',
    placeholder='Type something',
    description='Model:',
    disabled=False
)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

Sliders and information widgets

In [6]:
# Road Following sliders
speed_control_slider = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.01, description='speed control')
steering_gain_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.03, description='steering gain')
steering_kd_gain_slider = widgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.02, description='steering kd')
steering_bias_slider = widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

#Collision Avoidance sliders
blocked_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='blocked')
stop_duration_slider = widgets.IntSlider(min=1, max=1000, step=1, value=10, description='time for stop')
blocked_threshold = widgets.FloatSlider(min=0, max=1.0, step=0.01, value=0.8, description='blocked threshold')

# Steering sliders
diff_gain_limit_slider = widgets.FloatSlider(min=0.0, max=1.8, step=0.01, value=1.5, description='diff_gain_limit')
curve_min_val_slider = widgets.FloatSlider(min=0.0, max=0.25, step=0.01, value=0.08, description='curve_min_val')
alpha_factor_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.01, value=0.00, description='alpha_factor')
steering_correction_activation = widgets.FloatSlider(min=0.0, max=math.pi, step=0.05, value=0.25,
                                                     description='steering_correction_activation')

# Object Detection Sliders
xmax_threshold_slider = widgets.IntSlider(min=0, max=224, step=1, value=120, description='Xmax_threshold')
ymax_threshold_slider = widgets.IntSlider(min=0, max=224, step=1, value=45, description='Ymax_threshold')
score_threshold_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=0.3, description='Score_threshold')

sign_stop_slider = widgets.FloatSlider(min=0.0, max=1.0, orientation='horizontal', description='SIGN STOP SCORE')

toggle_exec_btn = widgets.ToggleButton(
    value=False,
    description='Start',
    disabled=False,
    button_style='success',
    tooltip='Description',
    icon='check'
)

snapshot_btn = widgets.Button(
    description='snapshot'
)


make snapshot

In [7]:
def take_snapshot(b=None):
    try:
        os.makedirs("snapshot")
    except FileExistsError:
        pass
    with open(os.path.join('snapshot', str(uuid1()) + '.jpg'), 'wb') as f:
        f.write(image_widget.value)


snapshot_btn.on_click(take_snapshot)

debug slider for speed

In [8]:
a_slider = widgets.FloatSlider(min=-math.pi, max=math.pi, step=0.01, description='a')
xy_out = widgets.Text()

Display the controlling widgets

Variables for slider controls

In [9]:
speed_control, steer_gain, steer_kd_gain, steer_bias = speed_control_slider.value, 0.0, 0.0, 0.0

Init Roboter Object

In [10]:
robot = Robot()

Global Vars

In [11]:
angle_last, stop_counter = 0.0, 0
can_drive = True
stop_counter_limit = 10
x, y = 0.0, 0.0
robot_is_stopped = True
sign_stop_active = False
ignore_stop_sign = False
state_sign_stop = 0 # 0 = nix, 1 = 1s vor tatsächlichem Stopp, 2 = Stopp, 3 = ignoriere Stoppschilder
time_ugly = 0 # timer wird von allen verwendet
sign_no_limit_active = False
sign_limit_active = False

calculate speed values

In [12]:
def calculate_speed(last_a: float, x_in: float, y_in: float) -> (float, float, float):
    global a_slider
    if robot_is_stopped:
        return 0.0, 0.0, 0.0
    a = math.atan2(x_in, y_in)
    a_slider.value = a
    xy_out.value = f"({x:.02f} {y:.02f})"
    pid = a * steer_gain + (a - last_a) * steer_kd_gain
    steer_val = pid + steer_bias
    left = max(min(speed_control + steer_val, 1.0), 0.0)
    right = max(min(speed_control - steer_val, 1.0), 0.0)
    return a, left, right

In [13]:
def get_collision_chance(image) -> float:
    collision = model_collision(image)
    collision_softmax = functional.softmax(collision, dim=1).flatten()
    return float(collision_softmax[0])

In [14]:
def get_road_direction(image) -> (float, float):
    model_xy = model_road(image).detach().float().cpu().numpy().flatten()
    model_x = model_xy[0]
    model_y = (0.5 - model_xy[1]) / 2.0
    return model_x, model_y

drive execute logic

In [15]:

class Executor:
    
    def __init__(self):
        self.frame_counter = 1
        self.activate_yolo = True 
        
    def changeFC(self):
        if self.frame_counter == 2:
            self.frame_counter = 1
        else:
            self.frame_counter = 2
    
    def execute(self, change):
        global angle_last, blocked_slider, robot, stop_counter, stop_counter_limit, can_drive, x, y, blocked_threshold, sign_stop_slider
        global speed_control, steer_gain, steer_kd_gain, steer_bias
        global xmax_threshold_slider, ymax_threshold_slider, score_threshold_slider
        global robot, sign_stop_active, ignore_stop_sign, state_sign_stop, time_ugly

        image = change['new']

        #############################################
        if self.frame_counter == 2:
            self.changeFC()
            if self.activate_yolo == True:
                output = Object_detector.detect(image)
                obj = -111
                for ooo in output:
                    if obj == -111:
                        obj = ooo
                    elif (ooo['ymax'] > obj['ymax']) and obj['label'] != 'kreuzung' and obj['label'] != 'sign_forbidden':
                        obj = ooo
                
                if obj != -111:
                    if obj['score'] > score_threshold_slider.value and obj['xmax'] > xmax_threshold_slider.value and obj['ymax'] > ymax_threshold_slider.value:
                        if obj['label'] == 'sign_stop' and ignore_stop_sign == False:
                            sign_stop_slider.value = obj['score']  
                            if state_sign_stop == 0:
                                time_ugly = time.time()
                                state_sign_stop = 1
                        elif obj['label'] == 'sign_limit':
                            speed_control_slider.value = 0.10
                        elif obj['label'] == 'sign_nolimit':
                            speed_control_slider.value = 0.14            
                if state_sign_stop == 1:
                    if time.time() - time_ugly > 1:
                        state_sign_stop = 2
                        sign_stop_active = True
                        robot.stop()
                        time_ugly = time.time()
                elif state_sign_stop == 2:
                    if time.time() - time_ugly > 3:
                        state_sign_stop = 3
                        sign_stop_active = False
                        ignore_stop_sign = True
                        robot.forward(0.11)
                        time_ugly = time.time()
                elif state_sign_stop == 3:
                    if time.time() - time_ugly > 2:
                        state_sign_stop = 0
                        ignore_stop_sign = False
            else:
                self.activate_yolo = True

        else:
            self.changeFC()
            # get slider values
            image_preproc = preprocess(image).to(device)
            steer_gain = steering_gain_slider.value
            steer_kd_gain = steering_kd_gain_slider.value
            steer_bias = steering_bias_slider.value
            stop_counter_limit = stop_duration_slider.value
            max_speed = speed_control_slider.value
            if sign_stop_active == False:
                prob_blocked = get_collision_chance(image_preproc)
                blocked_slider.value = prob_blocked

                if can_drive:
                    can_drive = prob_blocked <= blocked_threshold.value
                    if can_drive:
                        stop_counter = 0
                        x, y = get_road_direction(image_preproc)
                        speed_control = max_speed
                    else:
                        stop_counter += 1
                else:
                    stop_counter += 1
                    if stop_counter < stop_counter_limit:
                        x, y, speed_control = 0.0, 0.0, 0
                    else:
                        can_drive = True
                        stop_counter = 0

                angle, left, right = calculate_speed(angle_last, x, y)
                angle_last = angle

                robot.left_motor.value = left
                robot.right_motor.value = right

                speed_output.value = f'{left:.02f}/{right:.02f}'
            else: 
                speed_output.value = f'{0}/{0}'
#######################################
executor = Executor()
executor.execute({'new': camera.value})

In [16]:
def start_stop_toggle(obj):
    global robot_is_stopped
    if obj["new"]:
        toggle_exec_btn.button_style = 'warning'
        toggle_exec_btn.icon = 'warning'
        toggle_exec_btn.description = "Stop"
        camera.observe(executor.execute, names='value')
        robot_is_stopped = False
    else:
        toggle_exec_btn.button_style = 'success'
        toggle_exec_btn.icon = 'check'
        toggle_exec_btn.description = 'Start'
        camera.unobserve(executor.execute, names='value')
        robot_is_stopped = True
        time.sleep(0.1)
        robot.stop()


toggle_exec_btn.observe(start_stop_toggle, 'value')

Working Values:
- steering_gain 0.03
- steering_kd 0.06

In [17]:
display(speed_control_slider, steering_gain_slider, steering_kd_gain_slider, steering_bias_slider)

display(diff_gain_limit_slider, curve_min_val_slider, alpha_factor_slider, steering_correction_activation)

display(score_threshold_slider, xmax_threshold_slider, ymax_threshold_slider)


display(widgets.HBox([toggle_exec_btn, snapshot_btn]))

display(widgets.HBox([image_widget]))
display(speed_output)

display(widgets.HBox([blocked_slider, blocked_threshold, stop_duration_slider]))
display(a_slider, xy_out)
display(widgets.HBox([sign_stop_slider]))

FloatSlider(value=0.0, description='speed control', max=1.0, step=0.01)

FloatSlider(value=0.03, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.02, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=1.5, description='diff_gain_limit', max=1.8, step=0.01)

...

IntSlider(value=120, description='Xmax_threshold', max=224)

IntSlider(value=45, description='Ymax_threshold', max=224)

Text(value='0.00/0.00', description='(l,r):', disabled=True, placeholder='')

FloatSlider(value=0.0, description='a', max=3.141592653589793, min=-3.141592653589793, step=0.01)

Text(value='')

In [18]:
#camera.stop()